In [1]:
'''
何個か簡単な操作コードを書いておく。
'''

import tensorflow as tf

In [50]:
'''
基本コマンド
'''


# 定数テンソル（後から変更できない）
const_a = tf.constant(1, dtype=tf.int32, name='a') # スカラ
const_b = tf.constant([1,2,3], dtype=tf.int32, name='b') # ベクトル


# 変数テンソル（後から変更できる。パラメータとかはこれで指定する）
var_a = tf.Variable(1, dtype=tf.int32, name='hoge') # スカラ


# 代入テンソル
assign_a = tf.assign(var_a, var_a + const_a) # 第一引数に第二引数の結果を代入する


# 全ての変数を初期化する(計算を実行する前に、必ずやっておくこと)
tf.global_variables_initializer() # 通常はSessionを貼ってから「sess.run(tf.global_variables_initializer())」の形で実行する。


# プレースホルダー（様々な値を受けることができる「箱」のようなもの。値が未定な状態でグラフを構築し、実行時に具体的な値を指定することができる。入力データに使うことが多い）
a = tf.placeholder(dtype=tf.int32, name='a') # shapeも未指定の場合
placeholder_b = tf.placeholder(shape=(None, 2) ,dtype=tf.float32, name='placeholder_b') # データサイズは未定なので、Noneを指定。


# プレースホルダーに値を代入
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('[1,2]を代入する')
    print('a = ', sess.run(a, feed_dict={a:[[1,2]]}))
    
    print('[[1,2],[3,4]]を代入する')
    print('a = ', sess.run(a, feed_dict={a:[[1,2],[3,4]]}))
    
    

# 足し算と掛け算(スカラ)
a = tf.constant(2, name='a')
b = tf.constant(3, name='b')
c = tf.add(a, b) # 「a + b」と同じ
d = tf.multiply(a, b) # 「a * b」と同じ

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # print('c:',sess.run(c))
    # print('d:',sess.run(d))    


# 行列演算
a = tf.constant([[1,2], [3,4]], name='a')
b = tf.constant([[5,6],[7,8]], name='b')

c = tf.matmul(a, b) # 行列の掛け算

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # print('run_a \n', sess.run(a))
    # print('run_b \n', sess.run(b))
    # print('run_c \n', sess.run(c))

    


[1,2]を代入する
a =  [[1 2]]
[[1,2],[3,4]]を代入する
a =  [[1 2]
 [3 4]]


In [62]:
'''
学習情報の保存。
tensorflowでは、セッション中の情報は別セッションに引き継げない。
なので、セッション中で学習した変数（パラメータ）はファイル等に出力して保存しておく。
そのために、「Saver」を使う
'''

'''
a = tf.Variable(1, name='a')
b = tf.assign(a, a + 1)


# 変数の値を出力する
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(b))
    print(sess.run(b))
    
    # 変数「b」の値を保存する
    saver.save(sess, './saver/variable_b.ckpt')
'''

# ファイルから変数を復元
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # 復元!
    saver.restore(sess, save_path='./saver/variable_b.ckpt')
    print(sess.run(b))

INFO:tensorflow:Restoring parameters from ./saver/variable_b.ckpt
4


In [14]:
'''
データフローグラフの出力。
どのような計算をしようとしているのか、ということを確認するために出力する。

ただ、正直言ってこれだけだとどのような形になっているのかよくわからない...。
tensorboardを使ったほうがいいかも。
'''

a = tf.constant(1, dtype=tf.int32, name='a')
b = tf.constant(2, dtype=tf.int32, name='b')
c = a + b

graph = tf.get_default_graph()
print(graph.as_graph_def())

node {
  name: "hoge"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_FLOAT
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_FLOAT
        tensor_shape {
        }
        float_val: 1.0
      }
    }
  }
}
node {
  name: "hoge_1"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 1
      }
    }
  }
}
node {
  name: "hoge_2"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val: 1
      }
    }
  }
}
node {
  name: "foo"
  op: "Const"
  attr {
    key: "dtype"
    value {
      type: DT_INT32
    }
  }
  attr {
    key: "value"
    value {
      tensor {
        dtype: DT_INT32
        tensor_shape {
        }
        int_val:

In [1]:
'''
tensorboardを使ってグラフの可視化

下記のようにグラフをファイル出力した後、同一仮想環境＆ディレクトリに移動してターミナル上で「tensorboard --logdir=logs」を実行。(logsの部分はファイル出力したディレクトリ名)
すると、「http://0.0.0.0:6006」でtensorboardが確認できる。

'''


import tensorflow as tf


LOG_DIR = './logs' # logの出力先ディレクトリ

a = tf.constant(1, name='a')
b = tf.constant(2, name='b')
c = a + b

graph = tf.get_default_graph()
summary_writer = tf.summary.FileWriter(LOG_DIR, graph)
summary_writer.close()

In [3]:
'''
最急降下法の実装
 y = (x-1)^2を最小化する。（最終的にはx=1になるはずである）
'''


import tensorflow as tf

# パラメータは変数として定義
x = tf.Variable(0., name='x')

# パラメータを使って最小化したい関数
func = (x-1)**2

# optimizer(SGD)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)

# 
train_step = optimizer.minimize(func)

# train_stepを繰り返し実行する
n_epochs = 20
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(n_epochs):
        sess.run(train_step)
        
    print('x =', sess.run(x))


x = 0.988471
